### PIP install

In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=e44e1f29e074ef4cbbb83ecc9991cb39265f48a9d5ad8070662be150912b90b2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


### 모듈 임포트

In [2]:
import os
from zipfile import ZipFile
import re
import pickle
import random
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import math
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sklearn.metrics.pairwise import cosine_similarity

### 데이터 압축 해제

In [3]:
!unzip /content/drive/MyDrive/data/json_20230707_SBERT수정본.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: json_data2/processed_95456.pickle  
  inflating: json_data2/processed_95457.pickle  
  inflating: json_data2/processed_95458.pickle  
  inflating: json_data2/processed_95459.pickle  
  inflating: json_data2/processed_9546.pickle  
  inflating: json_data2/processed_95460.pickle  
  inflating: json_data2/processed_95461.pickle  
  inflating: json_data2/processed_95462.pickle  
  inflating: json_data2/processed_95463.pickle  
  inflating: json_data2/processed_95464.pickle  
  inflating: json_data2/processed_95465.pickle  
  inflating: json_data2/processed_95466.pickle  
  inflating: json_data2/processed_95467.pickle  
  inflating: json_data2/processed_95468.pickle  
  inflating: json_data2/processed_95469.pickle  
  inflating: json_data2/processed_9547.pickle  
  inflating: json_data2/processed_95470.pickle  
  inflating: json_data2/processed_95471.pickle  
  inflating: json_data2/processed_95472.pickle  
  inflating: json_data2/processed_9

### json_list 변환

In [4]:
folder_path = '/content/json_data2'

file_names = sorted(os.listdir(folder_path))

data_list = []
for file_name in file_names:
  if file_name.startswith("processed_") and file_name.endswith(".pickle"):
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
      data = pickle.load(file)
      data_list.append(data)

### text_list, keyword_list 변환

In [5]:
import json
import pickle

file_path = "/content/drive/MyDrive/ColabNotebooks/image_explain_text_alldata.json"

with open(file_path,'rb') as file:
  image_texts = pickle.load(file)


In [6]:
#print(image_texts)

In [7]:
keyword_list = [] #키워드 저장
text_list = [] #텍스트 저장
new_data_list = [] #data_list에서 정상만 추출
image_text_list = [] #이미지 문장 저장

In [8]:
print(len(data_list))

101473


In [9]:
for i in range(len(data_list)):

  keyword = data_list[i]['keyword']

  if len(keyword) == 0 or image_texts[i]==" ":
    continue

  text = ""

  for j in range(len(data_list[i]['form'])):
    if type(data_list[i]['form'][j]['text']) == str:
      text += data_list[i]['form'][j]['text'] + "\n"

  keyword_list.append(" ".join( [ k for k in keyword if not k.isdigit() ] ) )
  text_list.append(text)
  new_data_list.append(data_list[i])
  image_text_list.append(image_texts[i])

data_list = new_data_list

In [10]:
save_path = '/content'
file_name = 'data_list.pickle'

file_path = os.path.join(save_path, file_name)

with open(file_path, 'wb') as file:
    pickle.dump(data_list, file)

In [11]:
print(len(text_list))
print(len(data_list))
print(len(keyword_list))
print(len(image_text_list))

101468
101468
101468
101468


### pair 생성 함수

In [12]:
import random
import numpy as np

def weighted_shuffle(arr):
    arr = arr.copy()
    shuffled = []
    while arr:
        weights = np.arange(len(arr), 0, -1)
        weights = weights / np.sum(weights)
        choice = np.random.choice(len(arr), p=weights)
        shuffled.append(arr.pop(choice))
    return shuffled

def get_positive_pairs(text, keywords, image, n=4):
  pairs = []

  keywords = keywords.split()
  key_lists = []
  for _ in range(n):
    shuffled_keywords = weighted_shuffle(keywords)
    key_lists.append( ' '.join(shuffled_keywords[:]) )

  for i in range(n):
    if(i%4==0):
      #pairs.append( (text, image) )
      pass
    elif(i%4==1):
      pairs.append( (f"<text> {text}" , f"<keyword> {key_lists[i]}" ) )
    elif(i%4==2):
      pairs.append( (f"<image> {image}", f"<keyword> {key_lists[i]}" ) )
    else:
      pairs.append( (f"<image> {image}\n <text> {text}" , f"<keyword> {key_lists[i]}" ) ) # image+text <-> keyword pair는 항상 1개씩 존재

  return pairs

def get_negative_pairs(text, keywords, image, n=4):

  def get_negative_pair(text, keyword, prompt=""):
    keyword_set = set(keyword.split())
    negative_keyword = random.choice(keyword_list)
    while set(negative_keyword.split()).intersection(keyword_set):
      negative_keyword = random.choice(keyword_list)
    negative_keyword = negative_keyword.split()
    shuffled_keywords = weighted_shuffle(negative_keyword)
    return (f"{prompt} {text}", f"<keyword> {' '.join(shuffled_keywords)}" )


  def get_negative_pair_image(text, image): #관사 제외 이미지 문장 속 단어가 하나도 안겹치도록 뽑음
    image_set = set( [ word for word in image.split()[4:] if word not in ['a','the','of'] ] )

    negative_image = random.choice(image_text_list)
    negative_image_set = set( [ word for word in negative_image.split()[4:] if word not in ['a','the','of'] ] )

    while negative_image_set.intersection(image_set):
      negative_image = random.choice(image_text_list)
      negative_image_set = set( [ word for word in negative_image.split()[4:] if word not in ['a','the','of'] ] )

    return ( f"<text> {text}" , f"<image> {negative_image}" )

  pairs = []
  for i in range(n):
    if(i%4==0):
      pairs.append(get_negative_pair(text, keywords, '<text>'))
    elif(i%4==1):
      pairs.append(get_negative_pair(image, keywords, '<image>'))
    elif(i%4==2):
      pairs.append(get_negative_pair( f"<image> {image}\n <text> {text}" , keywords))
    else:
      #pairs.append(get_negative_pair_image(text, image))
      pass

  return pairs

### positive pairs, negative pairs 정의

In [13]:
all_pairs = []

for i in tqdm(range(len(text_list))):

  text = text_list[i]
  keyword = keyword_list[i]
  image = image_text_list[i]
  data = data_list[i]

  all_pairs.extend(  [(text,keyword,image,data, i )]  ) # i is index for data_list

100%|██████████| 101468/101468 [00:00<00:00, 644677.38it/s]


In [14]:
train_data, test_data = train_test_split(all_pairs, test_size=0.05, random_state=42)
print(len(train_data), len(test_data))
print(train_data[0])

96394 5074
('ミリ\nらめん\n', '사장 라멘 일본 레드 빨간색 강렬한 가게', 'slide or page of a chinese text with a red design', {'form': [{'text': 'ミリ\nらめん', 'font_id': '5504', 'font_size': '27.51560648756664', 'style': {'bold': False, 'italic': False, 'strikeout': False, 'underline': False}, 'linespace': '1.12', 'opacity': '255', 'rotate': '0', 'id': 0, 'sheet_size': (355, 204), 'words': []}], 'template_idx': 73401, 'sheet_url': 'https://file.miricanvas.com/template_sheet/2023/06/08/21/20/kqifh9xvhevse0uv/sheet.xml', 'thumbnail_url': 'https://file.miricanvas.com/template_thumb/2023/06/08/21/20/kpdqhrb5pit1261f/thumb.png', 'page_no': 0, 'title': '붉은 컬러의 강렬한 붓글씨의 라멘집', 'primary_colors': [[251, 250, 250], [187, 44, 13]], 'primary_color_weights': [0.515, 0.483], 'language': 'ja', 'keyword': ['사장', '라멘', '일본', '레드', '빨간색', '강렬한', '가게']}, 61101)


In [15]:
positive_pairs = []
negative_pairs = []

for text, keyword, image, data, i in tqdm(train_data):

  positive_pairs.extend(get_positive_pairs(text, keyword, image, 4))
  negative_pairs.extend(get_negative_pairs(text, keyword, image, 4))

100%|██████████| 96394/96394 [08:13<00:00, 195.38it/s]


In [16]:
with open("pos.pickle", 'wb') as file:
  pickle.dump(positive_pairs,file)

with open("neg.pickle", 'wb') as file:
  pickle.dump(negative_pairs,file)

In [17]:
with open("pos.pickle", 'rb') as file:
  positive_pairs = pickle.load(file)

with open("neg.pickle", 'rb') as file:
  negative_pairs = pickle.load(file)

###test Evaluator 정의

In [18]:
from sentence_transformers.evaluation import RerankingEvaluator

evaluation_data = []
for limit, (text, keyword, image, data, i) in tqdm(enumerate(test_data)):

  temp = {'query': f"<text> {text}" ,
          'positive' : [f"<keyword> {keyword}" ,f"<image> {image}" ],
          'negative' : []}

  for text2, keyword2, image2, data2, i2 in test_data:
    if(i==i2):
      continue
    temp['negative'].extend([f"<keyword> {keyword2}" ,f"<image> {image2}" ])

  if(len(temp['positive'])<=0):
    print(f"skip {i}")
    continue

  evaluation_data.extend([temp])

  if(limit==100):
    break

100it [00:00, 173.52it/s]


In [19]:
print(len(temp['positive']))
print(len(temp['negative']))

2
10146


In [ ]:
#evaluation_data[0]

In [21]:
evaluator = RerankingEvaluator(evaluation_data, mrr_at_k = 10 , name = 'sbert_except_image-text_pair', # len(test_data) * 2만큼의 개수 중 상위 50개에 있는지 확인
                               write_csv= True, batch_size= 64,  #mAP
                               show_progress_bar= True, use_batched_encoding= True)

### Concatenate하여 pairs, labels 정의

In [22]:
pairs = positive_pairs + negative_pairs
labels = [1]*len(positive_pairs) + [0]*len(negative_pairs)

In [23]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

model.tokenizer.vocab['<image>'] = model.tokenizer.vocab.pop('##･')
model.tokenizer.vocab['<text>'] = model.tokenizer.vocab.pop('##𩽾')
model.tokenizer.vocab['<keyword>'] = model.tokenizer.vocab.pop('##𩾌')

In [24]:
train_data_pair = list(zip(pairs, labels))
train_examples = [InputExample(texts=[data[0][0], data[0][1]], label=float(data[1])) for data in train_data_pair]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=256) #train batch 256 test 64
train_loss = losses.CosineSimilarityLoss(model)

num_epochs = 5
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

In [ ]:
model.fit(output_path="/content/drive/MyDrive/SBERT_alltask_except-image-text-pair_20230731",
          train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          evaluator=evaluator,
          evaluation_steps=1300, #100개 기준 eval당 6분
          checkpoint_save_total_limit=1,
          checkpoint_save_steps=1300,
          checkpoint_path = '/content/drive/MyDrive/SBERT_alltask_except-image-text-pair_20230731_checkpoint')

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2260 [00:00<?, ?it/s]

In [ ]:
model.save("/content/drive/MyDrive/SBERT_alltask_except-image-text-pair_20230731")

In [ ]:
negative_pairs_test = [data[0] for data in test_data if data[1] == 0]
positive_pairs_test = [data[0] for data in test_data if data[1] == 1]

In [ ]:
sample_num = len(negative_pairs_test)

sum = 0
for a, b in tqdm(random.sample(negative_pairs_test, sample_num)):
  a = model.encode(a)
  b = model.encode(b)
  score = cosine_similarity([a], [b])[0][0]
  sum += score
print("Negative:", sum / sample_num)
sum = 0
for a, b in tqdm(random.sample(positive_pairs_test, sample_num)):
  a = model.encode(a)
  b = model.encode(b)
  score = cosine_similarity([a], [b])[0][0]
  sum += score
print("\nPositive:", sum / sample_num)

100%|██████████| 4956/4956 [01:25<00:00, 57.80it/s]


Negative: 0.10940443540938706


100%|██████████| 4956/4956 [01:25<00:00, 58.25it/s]


Positive: 0.8050584014974191


In [ ]:
model = SentenceTransformer('/content/drive/MyDrive/SBERT_small')

In [ ]:
import torch
keyword_embedding_list = model.encode(keyword_list, convert_to_tensor=True).cpu()
embedding_list = model.encode(text_list, convert_to_tensor=True).cpu()
torch.save(keyword_embedding_list, '/content/keyword_embedding_list.pth')
torch.save(embedding_list, '/content/embedding_list.pth')

In [ ]:
with open('/content/data_list.pickle', 'wb') as file:
    pickle.dump(data_list, file)

In [ ]:
keyword_embedding_set = []
keyword_set = list(set(keyword_list))

for keyword in tqdm(keyword_set):
    keyword_embedding_set.append((keyword, model.encode(keyword)))

def get_top_keyword(sentence, top_k = 3):
    sentence_embedding = model.encode(sentence)
    keyword_score_list = []
    for keyword, keyword_embedding in keyword_embedding_set:
        score = cosine_similarity([sentence_embedding], [keyword_embedding])[0][0]
        keyword_score_list.append((keyword, score))
    keyword_score_list.sort(key=lambda x: x[1], reverse=True)
    return keyword_score_list[:top_k]

100%|██████████| 8841/8841 [01:12<00:00, 121.57it/s]


In [ ]:
#중간중간 ###### 오타가 있는 레입아웃에 대한 시험.
sentence = '''
와인
'''
get_top_keyword(sentence, 4)

[('술 칵테일 주점 포스터 인테리어용 와인 호프 아트웍 영수증', 0.8479806),
 ('술 칵테일 주점 포스터 인테리어용 와인 호프 영수증', 0.84519935),
 ('술 칵테일 주점 포스터 인테리어용 와인 호프 아트웍', 0.82365596),
 ('광고 흰색 간판 타원 생맥주 에어간판 칵테일 보리 홍보 달 맥주 와인색 원 로고 사진 반짝 라인 일러스트 별 세계맥주 가게',
  0.77433455)]

In [ ]:
sentence = '''
제목: 세계를 변화시킬 IT 기업에서 함께 일해보세요!

소제목: 혁신과 기술로 미래를 연결하는 IT 전문가를 찾습니다.

첫 번째 단락:

우리는 세계적으로 인정받는 IT 기업으로서 혁신과 기술 개발에 주력하고 있습니다.
우리의 목표는 사람들의 삶을 편리하게 만들고, 사회와 비즈니스 영역에서 긍정적인 변화를 이끄는 것입니다.
우리는 열정적이고 창의적인 IT 전문가를 찾고 있습니다.
두 번째 단락:

우리 회사에서는 최신 기술을 활용하여 다양한 프로젝트를 진행합니다.
클라우드 컴퓨팅, 인공지능, 빅데이터 분석 등의 분야에서 우수한 역량을 보유한 전문가들과 함께 일합니다.
우리는 협업과 지속적인 학습을 중요시하며, 다양한 기회를 제공합니다.
세 번째 단락:

우리는 직원들의 성장과 발전을 위해 노력합니다.
경력 개발 프로그램과 교육 기회, 학습 자료 등을 제공하여 전문 기술과 리더십을 강화합니다.
우수한 성과에는 공정하고 경쟁력 있는 보상과 인센티브가 제공됩니다.
마지막 단락:

우리는 다양성과 포용성을 존중하는 기업문화를 갖추고 있습니다.
팀워크와 즐거움을 추구하며, 긍정적인 근무 환경을 조성합니다.
우리와 함께 미래를 형성하고, 세계를 변화시킬 일에 참여해보세요!
'''
get_top_keyword(sentence, 10)

[('비즈니스 인포그래픽 블루 순환 로드맵 팀워크 5단계', 0.86609876),
 ('코딩 사회 비즈니스 세미나 사내 미래 심플 사업 타이포그라피 소프트웨어 추상적인 발표 단순 코드 회의 회사 기술 개발 학회 직장 분석 세션 개발자 데이터 기업 학술대회 컨퍼런스 4차 혁명 사전등록',
  0.8582806),
 ('워크샵 지적인 비즈니스 도형 프로세스 자유로운 파랑 심플 설명 조직 종이 화이트 인사팀 모던 팀빌딩 목록 일러스트 순환 반짝임 분할 안내 단색 표 프레젠테이션 플랫한 말풍선 하늘색 ppt 기업 아이콘 라인',
  0.85207224),
 ('소개 임직원 기념행사 비전선포 비즈니스 빛 기념자료 블루 테크 발표용 발표 기업 미래 회사비전 교육 신년회', 0.84933853),
 ('회사 미래산업 스타트업 로봇 직장', 0.8315833),
 ('비즈니스 심플 강조 보라 노랑 발표 모던 빅테크 회사원 깔끔 실적 회사 목록 일러스트 반짝임 분할 친근한 안내 프레임 표 신선한 플랫한 기업 소셜미디어 주황 라인',
  0.8301631),
 ('과제 소개 앱 회사 서비스 제안서 ppt 발표 어플 분홍색', 0.8300071),
 ('네이비 모노톤 회사연혁 인포그래픽 톤온톤 타임라인 로드맵 그레이', 0.8296357),
 ('프레젠테이션 멘토 취업 ppt z세대 대기업 컴퓨터', 0.8288825),
 ('워크샵 도형 파랑 심플 설명 조직 그라데이션 보라 직장인 미래적인 팀워크 모던 3d 팀빌딩 깔끔 부드러운 목록 일러스트 검정 안내 프레임 비교 프레젠테이션 플랫한 ppt 기업 그래프 아이콘 주황 라인 잔잔한',
  0.8197837)]

In [ ]:
sentence = '''
"지금 당신의 꿈과 가장 가까운 순간, 우리와 함께하세요! 놀라운 할인과 특별한 혜택이 여러분을 기다리고 있습니다. 지금 바로 저희 제품을 만나보세요, 최고의 선택이 될 것입니다!"
'''
get_top_keyword(sentence, 10)

[('라인아트 홍보 퍼플 보라 상반기 결산 쇼핑몰 연보라 회사 일러스트 보라색 귀여운 상반기결산 파스텔 그림 기업 할인 소셜미디어 마케팅 세일',
  0.8905397),
 ('상세페이지 반짝 심플 보라 재밌는 공지 회사 흰색 유머 텍스트 콘텐츠 기하학적인 공모전 안내 직사각형 회색조 표 정돈된 신선한 플랫한 눈알 기업 숏폼 라인',
  0.87655187),
 ('쿠폰 캐릭터 아고 형광펜 경품 노트 이사 이벤트 일러스트 이삿짐센터 소문내기 프로모션 귀여운 참여안내 흑백 공책 운송업 할인 이벤트 카드뉴스 깔끔한',
  0.8761876),
 ('쿠폰 it 행사 홍보 증정 에어드롭 엔터테인먼트 노랑색 파랑색 nft 포인트 사은품 선물 이벤트 기프티콘 토큰 안내 이벤트팝업 광고 사이버 코인',
  0.8713442),
 ('영업 sns 홍보 리본 당첨자 발표 공지 당첨 노란색 선물 이벤트 회사 일러스트 축하 귀여운 안내 하늘색 기업 소셜미디어 상자 마케팅',
  0.8704221),
 ('동영상 팔로우 비즈니스 도형 정사각형 홍보 애니메이션 파워풀 소상공인 분홍색 핑크 손 핸드폰 파랑색 화려한 이모티콘 댓글 선물 이모지 3d 이벤트 회사 활동적인 블루 제품 인스타그램 귀여운 프레임 러블리한 선물이벤트 말풍선 밝은 기업 아이콘 소셜미디어 마케팅 리뷰',
  0.8670144),
 ('도형 카드뉴스 지하철 자유로운 심플 강조 공지 업로드 모던 연두 광고판 깔끔 이벤트 목록 반짝임 검정 분할 안내 미니멀 회색 플랫한 사진 말풍선 네온 광고 마케팅 라인',
  0.86645186),
 ('만화 플랫 캐릭터 홍보 심플 상반기 키치 남색 결산 민트색 이벤트 소비 회사 흰색 일러스트 프로모션 쇼핑 외곽선 상반기결산 그리드 기업 할인 소셜미디어 마케팅 세일 라인',
  0.8601446),
 ('팔로우 디지털 비즈니스 정사각형 주황색 쿠폰 젊은 검정색 홍보 애니메이션 소상공인 심플 파랑색 재밌는 댓글 모던 영수증 이벤트 블랙 회사 블루 제품 미니멀 선물이벤트 네온 오렌지 기업 

In [ ]:
sentence = '''
제목: 안전한 수영을 위한 주의사항!

소제목: 모두가 즐거운 수영을 위해 지켜야 할 사항들

첫 번째 단락:

수영장을 이용하실 때에는 반드시 안전을 최우선으로 생각해야 합니다.
다음은 수영장 이용시 지켜야 할 중요한 주의사항입니다.
두 번째 단락:

수영 전에는 몸을 충분히 풀어주세요.
워밍업과 스트레칭을 통해 근육을 준비하고 부상을 예방하세요.
건강한 상태에서만 수영을 시작하세요.
세 번째 단락:

물놀이를 즐길 때에는 깊은 수영장에서는 반드시 보호자의 도움을 받아야 합니다.
신체적인 능력에 맞는 구명용품을 착용하고, 생명보호 기구의 위치를 알아두세요.
다른 수영객과 충돌하지 않도록 주의하세요.
네 번째 단락:

수영장 내부에서는 안전한 행동이 중요합니다.
풀에서 뛰어오르거나 미끄러지는 행동은 사고의 원인이 될 수 있으니 자제해주세요.
슬리퍼나 비닐 바닥에는 주의하고, 갈라진 타일 등 위험한 부분에서 멀리 떨어져서 수영하세요.
다섯 번째 단락:

세안을 위해 수영장 샤워 시설을 이용할 때에는 청결을 유지해주세요.
개인 위생을 지켜 모두가 건강하게 수영을 즐길 수 있도록 합시다.
마지막 단락:

안전한 수영은 모두의 책임입니다.
주위 환경과 다른 수영객을 배려하며 즐거운 수영을 즐기고, 안전에 최선을 다해주세요.
'''
get_top_keyword(sentence, 10)

[('정사각형 여름 여가 자유로운 가로 파랑 홍보 스포츠 심플 개인 노랑 수영강사 강습 수영장 모던 물놀이 활동적인 일러스트 텍스트 귀여운 프레임 취미 하늘색 소셜미디어 수영 물 라인',
  0.96792465),
 ('수영영법 운동 카드뉴스 수영카드뉴스 수영안전수칙 수영기초 수영장 수영교실 안전수칙 수영', 0.9671744),
 ('귀여운 정사각형 스포츠센터 여름 운동 그림 블루 시원한 일러스트 아기자기한 다이어트 수영 물 수영레슨 수영장 건강 아이스크림 물방울',
  0.9456906),
 ('여름 가로 분홍 심플 강조 건강 pop 민트 수영장 활동적인 시원한 일러스트 다이어트 친근한 미니멀 운동 플랫한 하늘색 아이콘 소셜미디어 수영 물 주의사항 라인 스포츠센터',
  0.9401487),
 ('개강 파도타기 해변 휴가 강습 취미 서핑 보드 스포츠 익스트림 수강 바다 레저 파도 교육 커리큘럼 물놀이', 0.93472046),
 ('여름 가로 분홍 심플 강조 건강 pop 화이트 수영장 활동적인 시원한 일러스트 텍스트 다이어트 친근한 미니멀 운동 플랫한 하늘색 소셜미디어 수영 물 주의사항 라인 스포츠센터',
  0.93234664),
 ('평영 유산소 배영 atoz 크롤 효과 수영 물 영법 안전수칙 접영', 0.92559016),
 ('에세이 수영장일러스트 분홍색 수영하는사람 수영장 수영 따뜻한 작가의말.귀여운북커버 아기자기한 귀여운 여름운동 운동 귀여운수영일러스트 취미 하늘색 수영일기 사람일러스트.작가사진 여름주제 생활',
  0.9242916),
 ('정사각형 여름 여가 젊은 홍보 스포츠 반짝 심플 개인 수영장 흰색 일러스트 귀여운 바다 단색 운동 체크 초록색 그림 취미 하늘색 소셜미디어 수영 물 심플한',
  0.92013216),
 ('카드뉴스 여가 스포츠 심플 설명 요트 자연적인 개인 키치 다이빙 모던 깔끔 일러스트 캘리그라피 귀여운 레저 파란 취미 하늘색 순수한 잔잔한',
  0.9162958)]

In [ ]:
sentence = """
Title: Safety Guidelines for Enjoying Swimming!

Subtitle: Important precautions to ensure a safe and enjoyable swimming experience.

First paragraph:

When using a swimming pool, safety should be your top priority.
Here are some important precautions to keep in mind while using the swimming pool.
Second paragraph:

Always warm up your body before swimming.
Perform warm-up exercises and stretches to prepare your muscles and prevent injuries.
Start swimming only when you are in good health.
Third paragraph:

When engaging in water activities, especially in deep pools, always seek the assistance of a guardian.
Wear appropriate flotation devices according to your swimming ability and be aware of the location of life-saving equipment.
Be cautious and avoid collisions with other swimmers.
Fourth paragraph:

Practice safe behavior within the swimming pool area.
Avoid jumping or diving into the pool and refrain from engaging in activities that may lead to accidents.
Be mindful of slippery surfaces, such as tiles or wet floors, and maintain a safe distance from hazardous areas while swimming.
Fifth paragraph:

Maintain cleanliness while using the shower facilities in the swimming pool for personal hygiene.
By practicing good hygiene, we can ensure a healthy swimming environment for everyone.
Final paragraph:

Ensuring a safe swimming experience is everyone's responsibility.
Let's be considerate of our surroundings, respect other swimmers, and prioritize safety to enjoy a pleasant swimming experience.
"""
get_top_keyword(sentence, 10)

[('수영영법 운동 카드뉴스 수영카드뉴스 수영안전수칙 수영기초 수영장 수영교실 안전수칙 수영', 0.92305064),
 ('개강 파도타기 해변 휴가 강습 취미 서핑 보드 스포츠 익스트림 수강 바다 레저 파도 교육 커리큘럼 물놀이', 0.9048885),
 ('카드뉴스 여가 스포츠 심플 설명 요트 자연적인 개인 키치 다이빙 모던 깔끔 일러스트 캘리그라피 귀여운 레저 파란 취미 하늘색 순수한 잔잔한',
  0.9042554),
 ('동영상 여름스포츠 스킨스쿠버 여름 수상레저스포츠 여가 젊은 자유로운 스포츠 심플 설명 스쿠버다이빙 파란색 동그라미 산뜻한 공지 다이빙 노란색 수상스키 일러스트 안전교육 바다 안내 레저 다이내믹 프레임 워터슬레이 윈드서핑 래프팅 맑은 바나나보트 썸머 그림 프레젠테이션 서핑 ppt 아이콘 해양레저 제트스키 해양 라인 안전수칙',
  0.9026129),
 ('여름 카드뉴스 바다 어린이 수중 수영장 수영 물 교육 영법 토끼', 0.8961527),
 ('평영 유산소 배영 atoz 크롤 효과 수영 물 영법 안전수칙 접영', 0.89262855),
 ('도형 카드뉴스 여가 젊은 클래식 스포츠 감성적인 심플 설명 요트 개인 파란색 줄 다이빙 연두 분할 레저 선 취미 하늘색 라인 섬세한',
  0.8812556),
 ('여름 썸머 방학 서핑 수영', 0.87515676),
 ('정사각형 여름 여가 자유로운 가로 파랑 홍보 스포츠 심플 개인 노랑 수영강사 강습 수영장 모던 물놀이 활동적인 일러스트 텍스트 귀여운 프레임 취미 하늘색 소셜미디어 수영 물 라인',
  0.8750982),
 ('안전 스킨스쿠버 여름 여가 해저 스포츠 설명 교육 스쿠버다이빙 공지 다이빙 수중탐사 바다 안내 레저 프레젠테이션 잠수 ppt 해양레저 수영 해양',
  0.8710122)]

In [ ]:
%cd /content/drive/MyDrive/model

/content/drive/MyDrive/model
